In [112]:
import numpy as np
import networkx.algorithms.shortest_paths.astar as astar

R = -0.04 # reward
g = 0.9 # discount factor
e = 0.00001 # convergence factor

def update_util(grd, g, R, e):
    i = 0 # discount index
    
    while True:
        d = 0
        gprev = grd.copy()
        for i in range(3):
            for j in range(4):
                # policy instructions
                nexti = 0
                nextj = 0
                error_posi = 0
                error_posj = 0

                if gprev[i,j][1] == 8 or gprev[i,j][1] == 0: continue
                elif gprev[i,j][1] == 1:
                    nexti = i
                    nextj = j+1
                    error_posi = 1
                elif gprev[i,j][1] == -1:
                    nexti = i
                    nextj = j-1
                    error_posi = 1
                elif gprev[i,j][1] == 2:
                    nexti = i-1
                    nextj = j
                    error_posj = 1
                
                # check obstacle
                uval_a, uval_b = 0.0, 0.0
                if (i+error_posi) > 2 or (j+error_posj) > 3 or ((i+error_posi) == 1 and (j+error_posj) == 1):
                    uval_a = gprev[i,j][0]
                else:
                    uval_a = gprev[i+error_posi,j+error_posj][0]
                
                if (i-error_posi) < 0 or (j-error_posj) < 0 or ((i-error_posi) == 1 and (j-error_posj) == 1):
                    uval_b = gprev[i,j][0]
                else:
                    uval_b = gprev[i-error_posi,j-error_posj][0]
               
                # find utility value
                grd[i,j][0] = (g**i) * R + (0.8 * gprev[nexti,nextj][0] + 0.1 * uval_a + 0.1 * uval_b)
                
                if abs(grd[i,j][0] - gprev[i,j][0]) > d:
                    d = abs(gprev[i,j][0] - grd[i,j][0])
                
        if d < e*(1 - g)/g: break
        else: i += 1
    
class Node:
    def __init__(self, i, j, g, f, parent = None, children = []):
        self.i = i
        self.j = j
        self.g = g
        self.f = f
        self.parent = parent
        self.children = children

In [113]:
"""Set the grid: each cell is of type [utility,policy:up|down|left|right]"""
# left = -1, right = 1, up = 2, down = -2, obstacle = 8, target = 0
grid = np.array([
    [[0.0,1],[0.0,1],[0.0,1],[1.0,0]],
    [[0.0,2],[0.0,8],[0.0,2],[-1.0,0]],
    [[0.0,2],[0.0,-1],[0.0,-1],[0.0,-1]]
])

print("Calculating utilities...")
update_util(grid, g, R, e)
print("Finished!")
print(grid)

Calculating utilities...
Finished!
[[[ 0.81273116  1.        ]
  [ 0.86835616  1.        ]
  [ 0.91835616  1.        ]
  [ 1.          0.        ]]

 [[ 0.76773116  2.        ]
  [ 0.          8.        ]
  [ 0.66520548  2.        ]
  [-1.          0.        ]]

 [[ 0.7221686   2.        ]
  [ 0.68166847 -1.        ]
  [ 0.643839   -1.        ]
  [ 0.42518966 -1.        ]]]


In [114]:
# Implement A*
visited = []
opened = []
start = Node(2, 0, 0, 0)
goal = None

opened.append(start)

while len(opened) != 0:
    maxf = opened[0].f
    max_pos = 0
    for pos in range(len(opened)):
        if opened[pos].f > maxf:
            maxf = opened[pos].f
            max_pos = pos
    
    current = opened.pop(max_pos)

    if grid[current.i,current.j][1] == 0:
        print('Goal node found!\n\n')
        print("Position: {:d},{:d}".format(current.i,current.j))
        break

    # find current node's neighbours from grid
    left = current.j - 1
    right = current.j + 1
    up = current.i - 1
    down = current.i + 1

    children = []

    if left >= 0 and grid[current.i,left][1] != 8:
        children.append((current.i,left))
    
    if right < 4 and grid[current.i,right][1] != 8:
        children.append((current.i,right))
    
    if up >= 0 and grid[up,current.j][1] != 8:
        children.append((up,current.j))
    
    if down < 3 and grid[down,current.j][1] != 8:
        children.append((down,current.j))
    
    current.children.clear()

    # check children validity
    for ch in children:
        isvisited = False
        for nd in visited:
            if ch == (nd.i,nd.j):
                isvisited = True
                break
        if isvisited: continue

        cost = 0.0
        h = 0.0
        if grid[ch[0],ch[1]][1] == 0:
            cost = grid[ch[0],ch[1]][0]
            h = 0
        else:
            cost = R
            h = grid[ch[0],ch[1]][0]
        g = current.g + cost
        f = g + h

        isopened = False
        for nd in opened:
            if ch == (nd.i,nd.j) and g < nd.g:
                isopened = True
                break
        if isopened: continue

        new_node = Node(ch[0], ch[1], g, f)
        new_node.parent = current
        current.children.append(new_node)
        opened.append(new_node)

    print('Step through {:d},{:d}'.format(current.i, current.j))
    visited.append(current)

Step through 2,0
Step through 1,0
Step through 0,0
Step through 0,1
Step through 0,2
Goal node found!


Position: 0,3
